In [58]:
from Bio import Phylo

tree2 = Phylo.read("tree2.new", "newick")

tree1 = Phylo.read("tree1.new", "newick")

def append_unique(splits, out):
    if out not in splits:
        splits.append(out.copy())
        #Not completely sure why, but appending out.copy, rather than just out is very important.
        #I'm pretty sure that its because of scope.
        #Either way, praise stack overflow

def df_numbering(clade, num_dic, splits):
    out = []
    if clade.is_preterminal():
        for i in clade.clades:
            out.append(num_dic[i.name])
        clade.name = out
        append_unique(splits, out)
        return out
    elif clade.is_terminal():
        out.append(num_dic[clade.name])
        return out
    else:
        for i in clade.clades:
            x = df_numbering(i,num_dic,splits)
            if len(out) == 0:
                out = x
            elif len(out)==1:
                if out[0] < max(x):
                    out.append(max(x))
                else:
                    out = [min(x),out[0]]
            elif len(out)==2:
                out[0]=min([min(out),min(x)])
                out[1]=max([max(out),max(x)])
        append_unique(splits, out)
        return out
    
    
    


def rfdist(tree_1, tree_2):
    #Root both trees at same leaf

    tree_1.root_with_outgroup("seq1")
    tree_2.root_with_outgroup("seq1")

    #Make a depth-first numbering of leaves in t1

    tree_1.ladderize()
    DFS = tree_1.get_terminals()

    #rename the leaves in t2 by the DF-numbering of t1
    t2_leaves = tree_2.get_terminals()

    df_numbers = {}
    c = 0
    for i in DFS:
        df_numbers[i.name]=c
        c+=1

    #names are updated in trees [name, DF numbering/interval in subtree]
    splits_1 = []
    splits_2=[]
    df_numbering(tree_2.clade, df_numbers,splits_2)
    df_numbering(tree_1.clade, df_numbers, splits_1)

    shared = 0

    for i in splits_1:
        for n in splits_2:
            if min(i) or min(n) == 0:
                None
            elif i == n:
                shared += 1
    

    return len(splits_1)+len(splits_2)-2-2*shared


rfdist(tree1, tree2)

14